## Interactive Visual Analytics with Folium

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using matplotlib and seaborn and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using Folium.

#### Objectives
This lab contains the following tasks:

 - TASK 1: Mark all launch sites on a map
 - TASK 2: Mark the success/failed launches for each site on the map
 - TASK 3: Calculate the distances between a launch site to its proximities
 
After completed the above tasks, you should be able to find some geographical patterns about launch sites.

In [1]:
import folium
import pandas as pd
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [2]:
import requests
import io

# URL of the dataset
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

# Fetch the dataset using requests
response = requests.get(URL)

# Ensure the request was successful
if response.status_code == 200:
    # Read the content into a pandas DataFrame
    spacex_csv_file = io.BytesIO(response.content)
    spacex_df = pd.read_csv(spacex_csv_file)
    print(spacex_df.head())  # Display the first few rows of the DataFrame
else:
    print(f"Failed to fetch the dataset. HTTP status code: {response.status_code}")


   Flight Number        Date Time (UTC) Booster Version  Launch Site  \
0              1  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1              2  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2              3  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3              4  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4              5  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  Payload Mass (kg)  \
0               Dragon Spacecraft Qualification Unit                0.0   
1  Dragon demo flight C1, two CubeSats,  barrel o...                0.0   
2                             Dragon demo flight C2+              525.0   
3                                       SpaceX CRS-1              500.0   
4                                       SpaceX CRS-2              677.0   

       Orbit         Customer        Landing Outcome  class        Lat  \
0        LEO           SpaceX  Failure   (

In [3]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
# Create the 'success' column based on the 'class' column
spacex_df['success'] = spacex_df['class'].map({1: 'Success', 0: 'Failure'})

spacex_df.dtypes

Launch Site     object
Lat            float64
Long           float64
class            int64
success         object
dtype: object

In [4]:
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
florida = [28.562302,-80.577356]

In [6]:
# We first need to create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

site_map = folium.Map(location=florida, zoom_start=12)

# TODO: Create and add folium.Circle and folium.Marker for each launch site on the site map

# Loop through each row in the DataFrame to add circles
for index, row in launch_sites_df.iterrows():
    # Get latitude and longitude
    lat = row['Lat']
    lon = row['Long']
    # Add a circle for the launch site
    circle = folium.Circle(
        location=[lat, lon], 
        radius=50,         
        color='red',          # Border color
        fill=True,            # Enable fill
        fill_color='red',     # Fill color
        fill_opacity=0.6      # Set fill opacity
        )
    site_map.add_child(circle)
    
    # Add a marker for the launch site
    marker = folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(row['Launch Site']),
        icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), 
                     html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'], )
    )
    site_map.add_child(marker)
# Display the map
site_map


Now, you can explore the map by zoom-in/out the marked areas , and try to answer the following questions:

 - Are all launch sites in proximity to the Equator line? No, equator is located more southern crossing the South American continent.
 - Are all launch sites in very close proximity to the coast? Yes, all are near the coast.

# We first need to create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.
#site_map2 = folium.Map(location=florida, zoom_start=17)

# TODO: Create and add folium.Circle and folium.Marker for each launch site on the site map

# Loop through each row in the DataFrame to add circles
for index, row in spacex_df.iterrows():
    # Get latitude and longitude
    lat = row['Lat']
    lon = row['Long']
    # Add a circle for the launch site
    circle = folium.Circle(
        location=[lat, lon], 
        radius=100,         
        color='blue',          # Border color
        fill=True,            # Enable fill
        fill_color='blue',     # Fill color
        fill_opacity=0.6      # Set fill opacity
        )
    site_map2.add_child(circle)
    
    # Add a marker for the launch site
    marker = folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(row['success'])
    )
    site_map2.add_child(marker)
# Display the map
site_map2


In [7]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class,success
46,KSC LC-39A,28.573255,-80.646895,1,Success
47,KSC LC-39A,28.573255,-80.646895,1,Success
48,KSC LC-39A,28.573255,-80.646895,1,Success
49,CCAFS SLC-40,28.563197,-80.576820,1,Success
50,CCAFS SLC-40,28.563197,-80.576820,1,Success
51,CCAFS SLC-40,28.563197,-80.576820,0,Failure
52,CCAFS SLC-40,28.563197,-80.576820,0,Failure
53,CCAFS SLC-40,28.563197,-80.576820,0,Failure
54,CCAFS SLC-40,28.563197,-80.576820,1,Success
55,CCAFS SLC-40,28.563197,-80.576820,0,Failure


Next, let's create markers for all launch records. If a launch was successful (class=1), then we use a green marker and if a launch was failed, we use a red marker (class=0)

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

Let's first create a MarkerCluster object

In [8]:
marker_cluster = MarkerCluster()

In [9]:
# TODO: Create a new column in spacex_df dataframe called marker_color to store the marker colors based on the class value
spacex_df['marker_color'] = spacex_df['class'].map({1: 'green', 0: 'red'})
spacex_df

,Launch Site,Lat,Long,class,success,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,Failure,red
1,CCAFS LC-40,28.562302,-80.577356,0,Failure,red
2,CCAFS LC-40,28.562302,-80.577356,0,Failure,red
3,CCAFS LC-40,28.562302,-80.577356,0,Failure,red
4,CCAFS LC-40,28.562302,-80.577356,0,Failure,red
5,CCAFS LC-40,28.562302,-80.577356,0,Failure,red
6,CCAFS LC-40,28.562302,-80.577356,0,Failure,red
7,CCAFS LC-40,28.562302,-80.577356,0,Failure,red
8,CCAFS LC-40,28.562302,-80.577356,0,Failure,red
9,CCAFS LC-40,28.562302,-80.577356,0,Failure,red


In [10]:
from folium.plugins import MarkerCluster


In [11]:
print(spacex_df[['Lat', 'Long', 'marker_color']].isnull().sum())


Lat             0
Long            0
marker_color    0
dtype: int64


In [12]:
# Definer kortet med angivelse af centrering og initialt zoomniveau
#site_map = folium.Map(location=florida, zoom_start=17)

# Initialiser funktion til at tilføje markeringer, her til farvemarkering af succes/ikke-succes
marker_cluster = MarkerCluster()

# Tilføj markering til kortet
site_map.add_child(marker_cluster)

# Loop gennem datasættet for at tilføje markering enkeltvis til kortet for rækkerne
for index, record in spacex_df.iterrows():
    # Hent længde- og breddegrad i hver record for punktet
    lat = record['Lat']
    lon = record['Long']
    # Hent farvemarkering for pågældende række
    marker_color = record['marker_color']  
    
    # Dan markeringen med placering ved koordinater og farvekategorisering
    marker = folium.Marker(
        location=[lat, lon], 
        icon=folium.Icon(color=marker_color, icon_color='white')
    )
    # Tilføj markeringen til clusteret af markeringer
    marker_cluster.add_child(marker)

# Display the map
site_map


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.

In [13]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map
#Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. 
# Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

In [14]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [15]:
# find coordinate of the closet coastline
launch_site_lat = 28.562302
launch_site_lon = -80.577356
coastline_lat =  28.56367  
coastline_lon = -80.56815
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_coastline

0.9121364983035082

In [16]:
kystlinjepunkt = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0), 
               html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),)
    )
site_map.add_child(kystlinjepunkt)

In [17]:
linje=folium.PolyLine(
    locations=[(launch_site_lat, launch_site_lon), (coastline_lat, coastline_lon)],
    color="blue")
site_map.add_child(linje)

In [18]:
# TODO: Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. 
# You need to use MousePosition to find the their coordinates on the map first
togspor_lat =  28.56262
togspor_lon = -80.58699
afstand_togbane = calculate_distance(launch_site_lat, launch_site_lon, togspor_lat, togspor_lon)
afstand_togbane

0.9418365095975598

In [20]:
togbanepunkt = folium.Marker(
    location=[togspor_lat, togspor_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0), 
               html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(afstand_togbane),)
    )
site_map.add_child(togbanepunkt)
linje_tog=folium.PolyLine(
    locations=[(launch_site_lat, launch_site_lon), (togspor_lat, togspor_lon)],
    color="blue")
site_map.add_child(linje_tog)